In [1]:
from torchvision import models
import torch

import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt

resnet18_pretrained = models.resnet18(pretrained=True)

print(resnet18_pretrained)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
num_classes= 2
num_ftrs = resnet18_pretrained.fc.in_features
resnet18_pretrained.fc = nn.Linear(num_ftrs, num_classes)



class TrainDataset(Dataset):
    def __init__(self, img_paths, transform, labels):
        self.img_paths = img_paths
        self.transform = transform
        self.labels = labels
        
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        
        label = self.labels[index]
        
        return image, label

    def __len__(self):
        return len(self.img_paths)



In [3]:

import os
female_lists = os.listdir('female')
img_female_list = []
for female_list in female_lists:
    img_female_list.append('female/' + female_list)
male_lists = os.listdir('male')
img_male_list = []
for male_list in male_lists:
    img_male_list.append('male/' + male_list)

import numpy as np

female_label = np.zeros(len(female_lists))
male_label = np.ones(len(male_lists))

female_male = np.concatenate((female_label,male_label), None)
img_paths = img_female_list + img_male_list

female_male = torch.tensor(female_male,dtype=torch.long)

In [4]:
female_male

tensor([0, 0, 0,  ..., 1, 1, 1])

In [5]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    transforms.RandomHorizontalFlip(0.5),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

dataset = TrainDataset(img_paths, transform, female_male)

loader = DataLoader(
    dataset,
    batch_size=200, # 
    shuffle=True # shuffle 추가
)
from sklearn.metrics import f1_score
device = torch.device('cuda')
model = resnet18_pretrained.to(device)
model.train()

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

for epoch in range(1):
    running_loss = 0.0
    epoch_f1 = 0
    n_iter = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # t_ = torch.zeros((100,18))
        # t_[range(100),labels]=1
        # print(t_)
        # t_ = t_.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:
            print('{},{:.5f} loss: {:.3f}'.format(epoch+1, i+1, running_loss/10))
            running_loss=0.0
        epoch_f1 += f1_score(preds.cpu().numpy(), labels.cpu().numpy(), average='macro')
        n_iter += 1
    print('Finished')
    print('{:.4f}'.format(epoch_f1))


1,10.00000 loss: 0.530
1,20.00000 loss: 0.185
1,30.00000 loss: 0.111
1,40.00000 loss: 0.105
1,50.00000 loss: 0.081
1,60.00000 loss: 0.074
1,70.00000 loss: 0.087
1,80.00000 loss: 0.091
1,90.00000 loss: 0.083
Finished
93.5453


In [12]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [13]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'gender_classi_resnet18_.csv'), index=False)
print('test inference is done!')

test inference is done!
